In [1]:
import numpy as np 
import pandas as pd 
import os 
import cv2 
import gc 
from tqdm import tqdm

In [2]:
files = os.listdir("./input/face-detection-data/dataset/")
files

['aishwarya_rai',
 'angelina_jolie',
 'arnold_schwarzenegger',
 'bhuvan_bam',
 'brad_pitt',
 'Courteney_Cox',
 'David_Schwimmer',
 'dhoni',
 'hardik_pandya',
 'Jennifer_Aniston',
 'Jim_Parsons',
 'Johnny_Galeck',
 'Kunal_Nayya',
 'Lisa_Kudrow',
 'manoj_bajpayee',
 'Matthew_Perry',
 'Matt_LeBlanc',
 'messi',
 'mohamed_ali',
 'Pankaj_Tripathi',
 'pewdiepie',
 'random_person',
 'ROHIT_SHARMA',
 'ronaldo',
 'Sachin_Tendulka',
 'scarlett_johansson',
 'Simon_Helberg',
 'suresh_raina',
 'sylvester_stallone',
 'virat_kohli']

In [3]:
image_array = []

label_array = []

In [4]:
image_array=[]  # it's a list later i will convert it to array
label_array=[]
path="./input/face-detection-data/dataset/"
# loop through each sub-folder in train
for i in range(len(files)):
    # files in sub-folder
    file_sub=os.listdir(path+files[i])

    for k in tqdm(range(len(file_sub))):
        try:
            img=cv2.imread(path+files[i]+"/"+file_sub[k])
            img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img=cv2.resize(img,(96,96))
            image_array.append(img)
            label_array.append(i)
        except:
            pass

100%|██████████| 391/391 [00:02<00:00, 145.05it/s]


In [5]:
import gc
gc.collect()

9

In [6]:
image_array = np.array(image_array)/255.0
label_array = np.array(label_array)

In [7]:
print(image_array[0])

[[[0.45490196 0.38823529 0.38039216]
  [0.25490196 0.18823529 0.18039216]
  [0.38823529 0.33333333 0.32156863]
  ...
  [0.52941176 0.5254902  0.50980392]
  [0.27058824 0.28235294 0.2627451 ]
  [0.25098039 0.25098039 0.24705882]]

 [[0.52941176 0.45490196 0.43529412]
  [0.4        0.3254902  0.30588235]
  [0.45098039 0.37647059 0.36078431]
  ...
  [0.30588235 0.30588235 0.29803922]
  [0.30980392 0.31764706 0.31372549]
  [0.25098039 0.24705882 0.26666667]]

 [[0.44705882 0.36862745 0.34117647]
  [0.40784314 0.31764706 0.28627451]
  [0.47843137 0.38431373 0.36862745]
  ...
  [0.36862745 0.36078431 0.36470588]
  [0.29411765 0.29803922 0.30980392]
  [0.24313725 0.23921569 0.26666667]]

 ...

 [[0.01176471 0.01176471 0.01176471]
  [0.01176471 0.01176471 0.01176471]
  [0.00784314 0.00784314 0.00784314]
  ...
  [0.05490196 0.05098039 0.07058824]
  [0.04313725 0.03921569 0.05882353]
  [0.05098039 0.04705882 0.06666667]]

 [[0.01176471 0.01176471 0.01176471]
  [0.01176471 0.01176471 0.01176471]


In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(image_array,label_array,test_size=0.15)

In [9]:
from keras import layers,callbacks,utils, applications, optimizers
from keras.models import Sequential, Model, load_model

In [10]:
len(files)

30

In [11]:
import tensorflow as tf

In [12]:
model = Sequential()
pretrained_model = tf.keras.applications.EfficientNetB0(input_shape=(96,96,3),include_top=False, weights="imagenet")
model.add(pretrained_model)
#im stupid i forgot pooling layers
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.3))
#add dense layers 
model.add(layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 3, 3, 1280)       4049571   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 4,050,852
Trainable params: 4,008,829
Non-trainable params: 42,023
_________________________________________________________________


In [13]:
model.compile(optimizer = "adam", loss="mean_squared_error",metrics=["mae"])

In [14]:
checkpoint = "trained_model/model"

In [15]:
model_checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint,monitor="val_mae",mode="auto",save_best_only=True,save_weigts_only=True)
reduce_lf=tf.keras.callbacks.ReduceLROnPlateau(factor=0.9, monitor="val_mae",mode="auto",cooldown=0,patience=5, verbose=1, mon_lr=1e-6)

In [16]:
#main
EPOCHS=300
BATCH_SIZE=64

history=model.fit(X_train,
                 Y_train,
                 validation_data=(X_test,Y_test),
                 batch_size=BATCH_SIZE,
                 epochs=EPOCHS,
                 callbacks=[model_checkpoint,reduce_lf ]
                 )

Epoch 1/300
212/212 [==============================] - ETA: 0s - loss: 53.1501 - mae: 5.4756INFO:tensorflow:Assets written to: trained_model\model\assets


C:\Users\josht\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
C:\Users\josht\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


212/212 [==============================] - 510s 2s/step - loss: 53.1501 - mae: 5.4756 - val_loss: 76.5722 - val_mae: 7.5662 - lr: 0.0010
Epoch 2/300
212/212 [==============================] - ETA: 0s - loss: 21.7467 - mae: 3.3222INFO:tensorflow:Assets written to: trained_model\model\assets


C:\Users\josht\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
C:\Users\josht\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


212/212 [==============================] - 488s 2s/step - loss: 21.7467 - mae: 3.3222 - val_loss: 76.6303 - val_mae: 7.5048 - lr: 0.0010
Epoch 3/300
212/212 [==============================] - ETA: 0s - loss: 15.4177 - mae: 2.6876INFO:tensorflow:Assets written to: trained_model\model\assets


...

C:\Users\josht\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
C:\Users\josht\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


212/212 [==============================] - 476s 2s/step - loss: 7.4818 - mae: 1.7788 - val_loss: 71.9066 - val_mae: 7.1377 - lr: 0.0010
Epoch 7/300
212/212 [==============================] - 402s 2s/step - loss: 7.6666 - mae: 1.7495 - val_loss: 83.9828 - val_mae: 7.8166 - lr: 0.0010
Epoch 8/300
212/212 [==============================] - 408s 2s/step - loss: 5.7965 - mae: 1.4948 - val_loss: 133.9656 - val_mae: 9.4911 - lr: 0.0010
Epoch 9/300
212/212 [==============================] - 405s 2s/step - loss: 4.9988 - mae: 1.4039 - val_loss: 148.2282 - val_mae: 10.1122 - lr: 0.0010
Epoch 10/300
212/212 [==============================] - 369s 2s/step - loss: 5.2370 - mae: 1.3976 - val_loss: 271.7807 - val_mae: 14.2079 - lr: 0.0010
Epoch 11/300
212/212 [==============================] - ETA: 0s - loss: 4.4984 - mae: 1.3258
Epoch 00011: ReduceLROnPlateau reducing learning rate to 0.0009000000427477062.
212/212 [==============================] - 360s 2s/step - loss: 4.4984 - mae: 1.3258 - val_lo

C:\Users\josht\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
C:\Users\josht\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


212/212 [==============================] - 418s 2s/step - loss: 4.4191 - mae: 1.2756 - val_loss: 60.7989 - val_mae: 6.4333 - lr: 9.0000e-04
Epoch 14/300
212/212 [==============================] - 360s 2s/step - loss: 4.2926 - mae: 1.2790 - val_loss: 73.4100 - val_mae: 7.4084 - lr: 9.0000e-04
Epoch 15/300
212/212 [==============================] - 367s 2s/step - loss: 4.7130 - mae: 1.3143 - val_loss: 75.0683 - val_mae: 7.2697 - lr: 9.0000e-04
Epoch 16/300
212/212 [==============================] - 386s 2s/step - loss: 4.4143 - mae: 1.2321 - val_loss: 156.2603 - val_mae: 10.4035 - lr: 9.0000e-04
Epoch 17/300
212/212 [==============================] - 399s 2s/step - loss: 3.7478 - mae: 1.1344 - val_loss: 78.5717 - val_mae: 7.2876 - lr: 9.0000e-04
Epoch 18/300
212/212 [==============================] - ETA: 0s - loss: 4.1800 - mae: 1.2040
Epoch 00018: ReduceLROnPlateau reducing learning rate to 0.0008100000384729356.
212/212 [==============================] - 397s 2s/step - loss: 4.1800 - 

KeyboardInterrupt: 